In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import hvplot.pandas

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ChrisClass")

from scipy.stats import linregress
from api_keys import bea_key

Upload the Landfall Data to the Dataframe and format it so that landfall locations can be pulled from coordinates

In [2]:
# Upload landfall_csv and fomrat columns with better column names
landfall_df = pd.read_csv("Resources/landfall.csv")
landfall_df = landfall_df.rename(columns={'Season':'Year','SS HWS':'Category at Landfall','Name':'Storm Name','#':'# in Season'})

# Drop unneeded columns
landfall_df = landfall_df.drop(['Time','RMW nm','States Affected','Date'], axis=1)

# Create lists to store states, counties, and coordinates.
counties = []
states = []
lat_lngs = []

# Convert Lat and Long to a string
landfall_df['Longitude'] = landfall_df['Longitude'].astype(str)
landfall_df['Latitude'] = landfall_df['Latitude'].astype(str)

# Add lattitude and longitude values to list
lat_lngs = landfall_df[['Latitude','Longitude']].values.tolist()

# Loop through lat_lng list to pull county and state names with geopy  
for lat_lng in lat_lngs:
    try:
        county = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        county = county.raw['address']['county']
        counties.append(county)
        state = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        state = state.raw['address']['state']
        states.append(state)
    # To handle errors if data isn't found
    except:
        counties.append('')
        states.append('')

# Add county and state names to dataframe and remove 'county/parish' from string
landfall_df['County'] = counties
landfall_df['State'] = states
landfall_df['County'] = landfall_df['County'].str.replace(' County','')
landfall_df['County'] = landfall_df['County'].str.replace(' Parish','')

# Dictionary to store State names and abb.
state_to_abbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA",
    "Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA",
    "Hawaii": "HI","Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA",
    "Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD",
    "Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO",
    "Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ",
    "New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH",
    "Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC",
    "South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT",
    "Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC"}

# Replace State Names with codes
landfall_df['State'] = landfall_df['State'].replace(state_to_abbrev)

# Save as a new csv to Resources
landfall_df.to_csv('Resources/formatted_landfall.csv',index=False)

# Re-Upload formated landfall to notebook
formatted_landfall_df = pd.read_csv("Resources/formatted_landfall.csv")

# Drop rows with empty columns
formatted_landfall_df = formatted_landfall_df.dropna()

# Show dataframe preview
formatted_landfall_df.head()

,# in Season,Year,Latitude,Longitude,Max Winds (kt),Category at Landfall,Central Prssure (mb),Storm Name,County,State
0,3,1983,29.1,-95.1,100,3,962,Alicia,Galveston,TX
1,10,1984,33.9,-78.0,95,2,979,Diana,Brunswick,NC
3,4,1985,29.6,-92.7,80,1,987,Danny,Cameron,LA
4,5,1985,30.4,-89.2,100,3,959,Elena,Harrison,MS
5,9,1985,35.2,-75.6,90,2,942,Gloria,Dare,NC


Upload the Housing Price Index county data as a new dataframe

In [3]:
#Upload HPI csv to dataframe
hpi_df = pd.read_csv('Resources/hpi_county.csv')

#Format 'year' and 'FIPS code' as string add forward 0 back to the 'FIPS code'
hpi_df['Year'] = hpi_df['Year'].astype(str)
hpi_df['FIPS code'] = hpi_df['FIPS code'].astype(str)
hpi_df['FIPS code'] = hpi_df['FIPS code'].apply(lambda x: x.zfill(5))

#Replace empty '.' with 'NaN
hpi_df.replace('.',"NaN",inplace=True)

#Drop unnecessary columns and format 'Annual Change(%)'
hpi_df = hpi_df.drop(['HPI with 1990 base','HPI with 2000 base'], axis=1)
hpi_df = hpi_df.rename(columns={'Annual Change (%)': 'Annual HPI Change (%)'})
hpi_df

,State,County,FIPS code,Year,Annual HPI Change (%),HPI
0,AL,Autauga,01001,1986,NaN,100
1,AL,Autauga,01001,1987,-1.94,98.06
2,AL,Autauga,01001,1988,2.57,100.58
3,AL,Autauga,01001,1989,4.32,104.92
4,AL,Autauga,01001,1990,-0.29,104.62
...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.4
100428,WY,Weston,56045,2020,4.36,233.13
100429,WY,Weston,56045,2021,4.93,244.62
100430,WY,Weston,56045,2022,6.87,261.41


Use an API call to pull income and jobs data from the Bureau of Economic Analysis for every county from 1980 to 2023

In [4]:
#Create variables for the API call to get Income data by county
base_url = 'https://apps.bea.gov/api/data'
tablename = 'CAINC1' #per capita income by county
linecode = '1'
year_range = '1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023'

#Create URL to Bureau of Economic Analysis for per capita income by county by year
income_url = f'{base_url}?&UserID={bea_key}&method=GetData&datasetname=Regional&TableName={tablename}&LineCode={linecode}&Year={year_range}&GeoFips=COUNTY&ResultFormat=json'
#Pull the response into the notebook
income_json = requests.get(income_url).json()

#Display the result as a dataframe with 'json_normalize()' using only relevant columns 
json_df = pd.json_normalize(income_json['BEAAPI']['Results']['Data'])
income_json_df = json_df[['GeoFips','GeoName','TimePeriod','DataValue']]
#Format columns to match 'hpi_df' dataframe so that it can be merged later.
income_json_df = income_json_df.rename(columns={'GeoFips':'FIPS code','TimePeriod':'Year','GeoName':'County & State','DataValue':'PerCapita Income'})

#Save new csv to avoid long run time of above code 
income_json_df.to_csv('Resources/income_data.csv',index=False)

In [5]:
#Create variables for the API call to get jobs data by county
tablename = 'CAINC30'
linecode = '240'

#Create URL to Bureau of Economic Analysis for per capita income by county by year
county_url = f'{base_url}?&UserID={bea_key}&method=GetData&datasetname=Regional&TableName={tablename}&LineCode={linecode}&Year={year_range}&GeoFips=COUNTY&ResultFormat=json'
#Pull the response into the notebook
county_json = requests.get(county_url).json()

#Display the result as a dataframe with 'json_normalize()' using only relevant columns 
json_df = pd.json_normalize(county_json['BEAAPI']['Results']['Data'])
jobs_json_df = json_df[['GeoFips','GeoName','TimePeriod','DataValue']]
#Format columns to match 'hpi_df' dataframe so that it can be merged later.
jobs_json_df = jobs_json_df.rename(columns={'GeoFips':'FIPS code','TimePeriod':'Year','GeoName':'County & State','DataValue':'Total Jobs'})

#Save new csv to avoid long run time of above code 
jobs_json_df.to_csv('Resources/jobs_data.csv',index=False)

Upload newly created datasets to the notebook and merge them together, then merge them with the HPI dataframe

In [6]:
#Re-Upload new income data csv to notebook
income_df = pd.read_csv('Resources/income_data.csv')
#Format 'year' and 'FIPS code' as string add forward 0 back to the 'FIPS code'
income_df['Year'] = income_df['Year'].astype(str)
income_df['FIPS code'] = income_df['FIPS code'].astype(str)
income_df['FIPS code'] = income_df['FIPS code'].apply(lambda x: x.zfill(5))
#Convert new 'PerCapita Income' column to a float and divide by 100 to get dollar value
income_df['PerCapita Income'] = income_df['PerCapita Income'].astype(float)
income_df['PerCapita Income'] = income_df['PerCapita Income']/100
income_df.head()

,FIPS code,County & State,Year,PerCapita Income
0,01001,"Autauga, AL",2004,13419.34
1,01001,"Autauga, AL",2002,11514.99
2,01001,"Autauga, AL",2010,18468.80
3,01001,"Autauga, AL",1998,9256.40
4,01001,"Autauga, AL",2013,19660.36


In [7]:
#Re-Upload new jobs data csv to notebook
jobs_df = pd.read_csv('Resources/jobs_data.csv')
#Format 'year' and 'FIPS code' as string add forward 0 back to the 'FIPS code'
jobs_df['Year'] = jobs_df['Year'].astype(str)
jobs_df['FIPS code'] = jobs_df['FIPS code'].astype(str)
jobs_df['FIPS code'] = jobs_df['FIPS code'].apply(lambda x: x.zfill(5))
#Convert new 'Total Jobs' column to a float 
#jobs_df['Total Jobs'] = jobs_df['Total Jobs'].astype(int)
jobs_df.head()

,FIPS code,County & State,Year,Total Jobs
0,01001,"Autauga, AL",2013,16475
1,01001,"Autauga, AL",2020,18615
2,01001,"Autauga, AL",1986,9410
3,01001,"Autauga, AL",1984,9305
4,01001,"Autauga, AL",1989,11165


In [8]:
#Merge income and jobs dataframes
bea_df = pd.merge(income_df,jobs_df,on=['FIPS code','Year','County & State'],how='outer')
bea_df

,FIPS code,County & State,Year,PerCapita Income,Total Jobs
0,01001,"Autauga, AL",1980,2500.67,11255
1,01001,"Autauga, AL",1981,2619.35,9636
2,01001,"Autauga, AL",1982,2801.88,9133
3,01001,"Autauga, AL",1983,3020.17,8975
4,01001,"Autauga, AL",1984,3374.59,9305
...,...,...,...,...,...
135015,56045,"Weston, WY",2018,3102.31,3884
135016,56045,"Weston, WY",2019,3160.01,3918
135017,56045,"Weston, WY",2020,3399.07,3912
135018,56045,"Weston, WY",2021,3365.03,3921


In [9]:
#Merge bea and hpi datasets
bea_hpi_df = pd.merge(hpi_df,bea_df,on = ['FIPS code','Year'],how='outer')
#Drop NaN values from 'HPI' and 'PerCapita Income'
bea_hpi_df = bea_hpi_df.dropna(subset=['HPI','PerCapita Income'])
bea_hpi_df

,State,County,FIPS code,Year,Annual HPI Change (%),HPI,County & State,PerCapita Income,Total Jobs
6,AL,Autauga,01001,1986,NaN,100,"Autauga, AL",3906.85,9410.0
7,AL,Autauga,01001,1987,-1.94,98.06,"Autauga, AL",4242.53,10177.0
8,AL,Autauga,01001,1988,2.57,100.58,"Autauga, AL",4656.42,10740.0
9,AL,Autauga,01001,1989,4.32,104.92,"Autauga, AL",5031.59,11165.0
10,AL,Autauga,01001,1990,-0.29,104.62,"Autauga, AL",5318.09,11400.0
...,...,...,...,...,...,...,...,...,...
143045,WY,Weston,56045,2018,-0.9,206.21,"Weston, WY",3102.31,3884.0
143046,WY,Weston,56045,2019,8.33,223.4,"Weston, WY",3160.01,3918.0
143047,WY,Weston,56045,2020,4.36,233.13,"Weston, WY",3399.07,3912.0
143048,WY,Weston,56045,2021,4.93,244.62,"Weston, WY",3365.03,3921.0


Find the values of each metric for the years surrounding each row

Then download the dataframe as a csv with the name 'bea_hpi_data.csv' and reupload it to notebook

In [12]:
#Get the years around eachrow in the dataframe wit the shift function
bea_hpi_df['Year -2'] = bea_hpi_df.groupby('State')['Year'].shift(2)
bea_hpi_df['Year -1'] = bea_hpi_df.groupby('State')['Year'].shift(1)
bea_hpi_df['Year +1'] = bea_hpi_df.groupby('State')['Year'].shift(-1)
bea_hpi_df['Year +2'] = bea_hpi_df.groupby('State')['Year'].shift(-2)
bea_hpi_df['Year +3'] = bea_hpi_df.groupby('State')['Year'].shift(-3)


#Put the values for Annual HPI chnages in the surrounding years into the same row
bea_hpi_df['Annual HPI Change (%) -1'] = bea_hpi_df.groupby('State')['Annual HPI Change (%)'].shift(1)
bea_hpi_df['Annual HPI Change (%) +1'] = bea_hpi_df.groupby('State')['Annual HPI Change (%)'].shift(-1)
bea_hpi_df['Annual HPI Change (%) +2'] = bea_hpi_df.groupby('State')['Annual HPI Change (%)'].shift(-2)
bea_hpi_df['Annual HPI Change (%) +3'] = bea_hpi_df.groupby('State')['Annual HPI Change (%)'].shift(-3)

#Put the values for 'PerCapita Income' in the surrounding years into the same row
bea_hpi_df['PerCapita Income -2'] = bea_hpi_df.groupby('State')['PerCapita Income'].shift(2)
bea_hpi_df['PerCapita Income -1'] = bea_hpi_df.groupby('State')['PerCapita Income'].shift(1)
bea_hpi_df['PerCapita Income +1'] = bea_hpi_df.groupby('State')['PerCapita Income'].shift(-1)
bea_hpi_df['PerCapita Income +2'] = bea_hpi_df.groupby('State')['PerCapita Income'].shift(-2)
bea_hpi_df['PerCapita Income +3'] = bea_hpi_df.groupby('State')['PerCapita Income'].shift(-3)

#Put the values for 'PerCapita Income' in the surrounding years into the same row
bea_hpi_df['Total Jobs -2'] = bea_hpi_df.groupby('State')['Total Jobs'].shift(2)
bea_hpi_df['Total Jobs -1'] = bea_hpi_df.groupby('State')['Total Jobs'].shift(1)
bea_hpi_df['Total Jobs +1'] = bea_hpi_df.groupby('State')['Total Jobs'].shift(-1)
bea_hpi_df['Total Jobs +2'] = bea_hpi_df.groupby('State')['Total Jobs'].shift(-2)
bea_hpi_df['Total Jobs +3'] = bea_hpi_df.groupby('State')['Total Jobs'].shift(-3)

#Caluclate the difference in income between years and convert to percentage
bea_hpi_df['Annual Income Change (%) -1'] = (bea_hpi_df['PerCapita Income -1']-bea_hpi_df['PerCapita Income -2'])/bea_hpi_df['PerCapita Income -2']*100
bea_hpi_df['Annual Income Change (%)'] = (bea_hpi_df['PerCapita Income']-bea_hpi_df['PerCapita Income -1'])/bea_hpi_df['PerCapita Income -1']*100
bea_hpi_df['Annual Income Change (%) +1'] = (bea_hpi_df['PerCapita Income +1']-bea_hpi_df['PerCapita Income'])/bea_hpi_df['PerCapita Income']*100
bea_hpi_df['Annual Income Change (%) +2'] = (bea_hpi_df['PerCapita Income +2']-bea_hpi_df['PerCapita Income +1'])/bea_hpi_df['PerCapita Income +1']*100
bea_hpi_df['Annual Income Change (%) +3'] = (bea_hpi_df['PerCapita Income +3']-bea_hpi_df['PerCapita Income +2'])/bea_hpi_df['PerCapita Income +2']*100

#Caluclate the difference in income between years and convert to percentage
bea_hpi_df['Annual Jobs Change (%) -1'] = (bea_hpi_df['Total Jobs -1']-bea_hpi_df['Total Jobs -2'])/bea_hpi_df['Total Jobs -2']*100
bea_hpi_df['Annual Jobs Change (%)'] = (bea_hpi_df['Total Jobs']-bea_hpi_df['Total Jobs -1'])/bea_hpi_df['Total Jobs -1']*100
bea_hpi_df['Annual Jobs Change (%) +1'] = (bea_hpi_df['Total Jobs +1']-bea_hpi_df['Total Jobs'])/bea_hpi_df['Total Jobs']*100
bea_hpi_df['Annual Jobs Change (%) +2'] = (bea_hpi_df['Total Jobs +2']-bea_hpi_df['Total Jobs +1'])/bea_hpi_df['Total Jobs +1']*100
bea_hpi_df['Annual Jobs Change (%) +3'] = (bea_hpi_df['Total Jobs +3']-bea_hpi_df['Total Jobs +2'])/bea_hpi_df['Total Jobs +2']*100

#Replace all 'inf' calues with 'NaN'
bea_hpi_df.replace([np.inf, -np.inf], np.nan, inplace=True)

#Save combined hpi and income df to directory
bea_hpi_df.to_csv('Output_CSVs/bea_hpi_data.csv',index=False)

#Re-upload hpi csv to notebook
bea_hpi_df_csv = pd.read_csv('Output_CSVs/bea_hpi_data.csv')

#Convert 'FIPS code' to string again and add leading 0 back
bea_hpi_df_csv['FIPS code'] = bea_hpi_df_csv['FIPS code'].astype(str)
bea_hpi_df_csv['PerCapita Income'] = bea_hpi_df_csv['PerCapita Income'].astype(float)
bea_hpi_df_csv['FIPS code'] = bea_hpi_df_csv['FIPS code'].apply(lambda x: x.zfill(5))
#Drop any columns with NaN values
bea_hpi_df_csv = bea_hpi_df_csv.dropna()

# Display dataframe
bea_hpi_df_csv.head()

,State,County,FIPS code,Year,Annual HPI Change (%),HPI,County & State,PerCapita Income,Total Jobs,Year -2,...,Annual Income Change (%) -1,Annual Income Change (%),Annual Income Change (%) +1,Annual Income Change (%) +2,Annual Income Change (%) +3,Annual Jobs Change (%) -1,Annual Jobs Change (%),Annual Jobs Change (%) +1,Annual Jobs Change (%) +2,Annual Jobs Change (%) +3
6,AL,Autauga,01001,1986,NaN,100,"Autauga, AL",3906.85,9410.0,NaN,...,NaN,NaN,8.592088,9.755735,8.057048,NaN,NaN,8.150903,5.532082,3.957169
7,AL,Autauga,01001,1987,-1.94,98.06,"Autauga, AL",4242.53,10177.0,NaN,...,NaN,8.592088,9.755735,8.057048,5.694025,NaN,8.150903,5.532082,3.957169,2.104792
8,AL,Autauga,01001,1988,2.57,100.58,"Autauga, AL",4656.42,10740.0,1986,...,8.592088,9.755735,8.057048,5.694025,8.128294,8.150903,5.532082,3.957169,2.104792,3.219298
9,AL,Autauga,01001,1989,4.32,104.92,"Autauga, AL",5031.59,11165.0,1987,...,9.755735,8.057048,5.694025,8.128294,6.743230,5.532082,3.957169,2.104792,3.219298,2.821450
10,AL,Autauga,01001,1990,-0.29,104.62,"Autauga, AL",5318.09,11400.0,1988,...,8.057048,5.694025,8.128294,6.743230,6.514047,3.957169,2.104792,3.219298,2.821450,3.504422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143045,WY,Weston,56045,2018,-0.9,206.21,"Weston, WY",3102.31,3884.0,2016,...,-1.690128,4.128123,1.859904,7.565166,-1.001450,-2.352067,-1.521298,0.875386,-0.153139,0.230061
143046,WY,Weston,56045,2019,8.33,223.4,"Weston, WY",3160.01,3918.0,2017,...,4.128123,1.859904,7.565166,-1.001450,3.942015,-1.521298,0.875386,-0.153139,0.230061,3.672533
143047,WY,Weston,56045,2020,4.36,233.13,"Weston, WY",3399.07,3912.0,2018,...,1.859904,7.565166,-1.001450,3.942015,NaN,0.875386,-0.153139,0.230061,3.672533,NaN
143048,WY,Weston,56045,2021,4.93,244.62,"Weston, WY",3365.03,3921.0,2019,...,7.565166,-1.001450,3.942015,NaN,NaN,-0.153139,0.230061,3.672533,NaN,NaN


Create a series of aggrigate tables to determine the mean by:

    HPI, Income, and Jobs by:

        Year and nation, and year and state



In [14]:
#Create an aggregate table to determine mean by year and by nation for HPI
by_nation_hpi = bea_hpi_df_csv.groupby(['Year'])
hpi_change_by_nation = by_nation_hpi['Annual HPI Change (%)'].agg(['mean'])
hpi_change_by_nation.head()

#Create lists to store the mean values for the surrounding years
prev_mean_national = []
mean_national_1 = []
mean_national_2 = []
mean_national_3 = []

#Use for loops to get the value for the mean in the year prior and the three years following
for index in hpi_change_by_nation.index:
    try:
        value = hpi_change_by_nation.at[index-1,'mean']
        prev_mean_national.append(value)
    except:
        prev_mean_national.append('NaN')

for index in hpi_change_by_nation.index:
    try:
        value = hpi_change_by_nation.at[index+1,'mean']
        mean_national_1.append(value)
    except:
        mean_national_1.append('NaN')

for index in hpi_change_by_nation.index:
    try:
        value = hpi_change_by_nation.at[index+2,'mean']
        mean_national_2.append(value)
    except:
        mean_national_2.append('NaN')

for index in hpi_change_by_nation.index:
    try:
        value = hpi_change_by_nation.at[index+3,'mean']
        mean_national_3.append(value)
    except:
        mean_national_3.append('NaN')

#Append hpi mean dataframe with the surrounding year values in the same rows
hpi_change_by_nation['National HPI AC Mean -1'] = prev_mean_national
hpi_change_by_nation['National HPI AC Mean +1'] = mean_national_1
hpi_change_by_nation['National HPI AC Mean +2'] = mean_national_2
hpi_change_by_nation['National HPI AC Mean +3'] = mean_national_3
hpi_change_by_nation = hpi_change_by_nation.rename(columns={'mean':'National HPI AC Mean'})
hpi_change_by_nation.head()

,National HPI AC Mean,National HPI AC Mean -1,National HPI AC Mean +1,National HPI AC Mean +2,National HPI AC Mean +3
Year,,,,,
1980,7.934871,NaN,4.650665,0.669119,7.281257
1981,4.650665,7.934871,0.669119,7.281257,3.68516
1982,0.669119,4.650665,7.281257,3.68516,3.419117
1983,7.281257,0.669119,3.68516,3.419117,4.337633
1984,3.685160,7.281257,3.419117,4.337633,3.50874


In [15]:
#Create an aggregate table to determine mean by year and by state for HPI
by_state_hpi = bea_hpi_df_csv.groupby(['State','Year'])
hpi_change_by_state = by_state_hpi['Annual HPI Change (%)'].agg(['mean'])
hpi_change_by_state = hpi_change_by_state.reset_index(level=1)

#Use shift function to get the values of the surrounding years into the same row
hpi_change_by_state['State HPI AC Mean -1'] = hpi_change_by_state.groupby('State')['mean'].shift(1)
hpi_change_by_state['State HPI AC Mean +1'] = hpi_change_by_state.groupby('State')['mean'].shift(-1)
hpi_change_by_state['State HPI AC Mean +2'] = hpi_change_by_state.groupby('State')['mean'].shift(-2)
hpi_change_by_state['State HPI AC Mean +3'] = hpi_change_by_state.groupby('State')['mean'].shift(-3)
hpi_change_by_state = hpi_change_by_state.rename(columns={'mean':'State HPI AC Mean'})
hpi_change_by_state.head()

,Year,State HPI AC Mean,State HPI AC Mean -1,State HPI AC Mean +1,State HPI AC Mean +2,State HPI AC Mean +3
State,,,,,,
AK,1982,12.500,NaN,8.110000,4.180000,-5.760000
AK,1983,8.110,12.50,4.180000,-5.760000,-5.134000
AK,1984,4.180,8.11,-5.760000,-5.134000,-16.603333
AK,1985,-5.760,4.18,-5.134000,-16.603333,5.870000
AK,1986,-5.134,-5.76,-16.603333,5.870000,-6.528333


In [16]:
#Create an aggregate table to determine mean by year by state and by nation for income
by_nation_income = bea_hpi_df_csv.groupby(['Year'])
income_change_by_nation = by_nation_income['Annual Income Change (%)'].agg(['mean'])
income_change_by_nation.head()

#Create lists to store the mean values for the surrounding years
prev_mean_income_national = []
mean_income_national_1 = []
mean_income_national_2 = []
mean_income_national_3 = []

#Use for loops to get the value for the mean in the year prior and the three years following
for index in income_change_by_nation.index:
    try:
        value = income_change_by_nation.at[index-1,'mean']
        prev_mean_income_national.append(value)
    except:
        prev_mean_income_national.append('NaN')

for index in income_change_by_nation.index:
    try:
        value = income_change_by_nation.at[index+1,'mean']
        mean_income_national_1.append(value)
    except:
        mean_income_national_1.append('NaN')

for index in income_change_by_nation.index:
    try:
        value = income_change_by_nation.at[index+2,'mean']
        mean_income_national_2.append(value)
    except:
        mean_income_national_2.append('NaN')

for index in income_change_by_nation.index:
    try:
        value = income_change_by_nation.at[index+3,'mean']
        mean_income_national_3.append(value)
    except:
        mean_income_national_3.append('NaN')

#Append hpi mean dataframe with the surrounding year values in the same rows
income_change_by_nation['National Income AC Mean -1'] = prev_mean_income_national
income_change_by_nation['National Income AC Mean +1'] = mean_income_national_1
income_change_by_nation['National Income AC Mean +2'] = mean_income_national_2
income_change_by_nation['National Income AC Mean +3'] = mean_income_national_2
income_change_by_nation = income_change_by_nation.rename(columns={'mean':'National Income AC Mean'})
income_change_by_nation.head()

,National Income AC Mean,National Income AC Mean -1,National Income AC Mean +1,National Income AC Mean +2,National Income AC Mean +3
Year,,,,,
1980,64.336183,NaN,12.493676,7.332994,7.332994
1981,12.493676,64.336183,7.332994,7.169918,7.169918
1982,7.332994,12.493676,7.169918,10.232273,10.232273
1983,7.169918,7.332994,10.232273,6.855398,6.855398
1984,10.232273,7.169918,6.855398,5.434463,5.434463


In [17]:
#Create an aggregate table to determine mean by year and by state for income
by_state_income = bea_hpi_df_csv.groupby(['State','Year'])
income_change_by_state = by_state_income['Annual Income Change (%)'].agg(['mean'])
income_change_by_state = income_change_by_state.reset_index(level=1)

#Use shift function to get the values of the surrounding years into the same row
income_change_by_state['State Income AC Mean -1'] = income_change_by_state.groupby('State')['mean'].shift(1)
income_change_by_state['State Income AC Mean +1'] = income_change_by_state.groupby('State')['mean'].shift(-1)
income_change_by_state['State Income AC Mean +2'] = income_change_by_state.groupby('State')['mean'].shift(-2)
income_change_by_state['State Income AC Mean +3'] = income_change_by_state.groupby('State')['mean'].shift(-3)
income_change_by_state = income_change_by_state.rename(columns={'mean':'State Income AC Mean'})
income_change_by_state.head()


,Year,State Income AC Mean,State Income AC Mean -1,State Income AC Mean +1,State Income AC Mean +2,State Income AC Mean +3
State,,,,,,
AK,1982,23.763363,NaN,9.710968,8.277082,6.293637
AK,1983,9.710968,23.763363,8.277082,6.293637,-0.609627
AK,1984,8.277082,9.710968,6.293637,-0.609627,-5.678111
AK,1985,6.293637,8.277082,-0.609627,-5.678111,5.215518
AK,1986,-0.609627,6.293637,-5.678111,5.215518,9.486219


In [18]:
#Create an aggregate table to determine mean by year by state and by nation for income
by_nation_jobs = bea_hpi_df_csv.groupby(['Year'])
jobs_change_by_nation = by_nation_jobs['Annual Jobs Change (%)'].agg(['mean'])
jobs_change_by_nation.head()

#Create lists to store the mean values for the surrounding years
prev_mean_jobs_national = []
mean_jobs_national_1 = []
mean_jobs_national_2 = []
mean_jobs_national_3 = []

#Use for loops to get the value for the mean in the year prior and the three years following
for index in jobs_change_by_nation.index:
    try:
        value = jobs_change_by_nation.at[index-1,'mean']
        prev_mean_jobs_national.append(value)
    except:
        prev_mean_jobs_national.append('NaN')

for index in jobs_change_by_nation.index:
    try:
        value = jobs_change_by_nation.at[index+1,'mean']
        mean_jobs_national_1.append(value)
    except:
        mean_jobs_national_1.append('NaN')

for index in jobs_change_by_nation.index:
    try:
        value = jobs_change_by_nation.at[index+2,'mean']
        mean_jobs_national_2.append(value)
    except:
        mean_jobs_national_2.append('NaN')

for index in jobs_change_by_nation.index:
    try:
        value = jobs_change_by_nation.at[index+3,'mean']
        mean_jobs_national_3.append(value)
    except:
        mean_jobs_national_3.append('NaN')

#Append hpi mean dataframe with the surrounding year values in the same rows
jobs_change_by_nation['National Jobs AC Mean -1'] = prev_mean_jobs_national
jobs_change_by_nation['National Jobs AC Mean +1'] = mean_jobs_national_1
jobs_change_by_nation['National Jobs AC Mean +2'] = mean_jobs_national_2
jobs_change_by_nation['National Jobs AC Mean +3'] = mean_jobs_national_3
jobs_change_by_nation = jobs_change_by_nation.rename(columns={'mean':'National Jobs AC Mean'})
jobs_change_by_nation.head()

,National Jobs AC Mean,National Jobs AC Mean -1,National Jobs AC Mean +1,National Jobs AC Mean +2,National Jobs AC Mean +3
Year,,,,,
1980,731.406152,NaN,1.610035,-0.126481,1.69622
1981,1.610035,731.406152,-0.126481,1.69622,4.441057
1982,-0.126481,1.610035,1.69622,4.441057,2.850772
1983,1.696220,-0.126481,4.441057,2.850772,1.878807
1984,4.441057,1.69622,2.850772,1.878807,3.008315


In [19]:
#Create an aggregate table to determine mean by year and by state for income
by_state_jobs = bea_hpi_df_csv.groupby(['State','Year'])
jobs_change_by_state = by_state_jobs['Annual Jobs Change (%)'].agg(['mean'])
jobs_change_by_state = jobs_change_by_state.reset_index(level=1)

#Use shift function to get the values of the surrounding years into the same row
jobs_change_by_state['State Jobs AC Mean -1'] = jobs_change_by_state.groupby('State')['mean'].shift(1)
jobs_change_by_state['State Jobs AC Mean +1'] = jobs_change_by_state.groupby('State')['mean'].shift(-1)
jobs_change_by_state['State Jobs AC Mean +2'] = jobs_change_by_state.groupby('State')['mean'].shift(-2)
jobs_change_by_state['State Jobs AC Mean +3'] = jobs_change_by_state.groupby('State')['mean'].shift(-3)
jobs_change_by_state = jobs_change_by_state.rename(columns={'mean':'State Jobs AC Mean'})
jobs_change_by_state.head()

,Year,State Jobs AC Mean,State Jobs AC Mean -1,State Jobs AC Mean +1,State Jobs AC Mean +2,State Jobs AC Mean +3
State,,,,,,
AK,1982,12.184209,NaN,7.683151,4.526859,1.687466
AK,1983,7.683151,12.184209,4.526859,1.687466,-3.015252
AK,1984,4.526859,7.683151,1.687466,-3.015252,1.332928
AK,1985,1.687466,4.526859,-3.015252,1.332928,3.795378
AK,1986,-3.015252,1.687466,1.332928,3.795378,4.339341


Merge all the newly created aggregate tables to the main dataframe 'bea_hpi_df_csv'

Save it as a new csv titled: 'aggregated_bea_hpi_data.csv'

In [20]:
#Merge aggregate data frames above with the main dataset
merged_df = pd.merge(bea_hpi_df_csv,hpi_change_by_nation,on = ['Year'],how='outer')
merged_df2 = pd.merge(merged_df,hpi_change_by_state,on=['Year','State'],how='outer')
merged_df3 = pd.merge(merged_df2,income_change_by_nation,on=['Year'],how='outer')
merged_df4 = pd.merge(merged_df3,income_change_by_state,on=['Year','State'],how='outer')
merged_df5 = pd.merge(merged_df4,jobs_change_by_nation,on=['Year'],how='outer')
aggregated_bea_hpi_df = pd.merge(merged_df5,jobs_change_by_state,on=['Year','State'],how='outer')

#Drop outer years to get rid of misaligned rows
aggregated_bea_hpi_df = aggregated_bea_hpi_df[(aggregated_bea_hpi_df['Year'] < 2020) & (aggregated_bea_hpi_df['Year'] > 1982)]

#Save Merged df to directory
aggregated_bea_hpi_df.to_csv('Output_CSVs/aggregated_bea_hpi_data.csv',index=False)

aggregated_bea_hpi_df

,State,County,FIPS code,Year,Annual HPI Change (%),HPI,County & State,PerCapita Income,Total Jobs,Year -2,...,National Jobs AC Mean,National Jobs AC Mean -1,National Jobs AC Mean +1,National Jobs AC Mean +2,National Jobs AC Mean +3,State Jobs AC Mean,State Jobs AC Mean -1,State Jobs AC Mean +1,State Jobs AC Mean +2,State Jobs AC Mean +3
2000,AK,Anchorage,02020,1983,8.11,225.95,"Anchorage Municipality, AK",46518.69,142019.0,1981.0,...,1.696220,-0.126481,4.441057,2.850772,1.878807,7.683151,12.184209,4.526859,1.687466,-3.015252
2001,AL,Baldwin,01003,1983,30.91,138.04,"Baldwin, AL",8446.81,31306.0,1981.0,...,1.696220,-0.126481,4.441057,2.850772,1.878807,2.543453,-1.074323,5.399666,2.684238,2.371115
2002,AL,Calhoun,01015,1983,10.84,144.33,"Calhoun, AL",12193.67,56216.0,1981.0,...,1.696220,-0.126481,4.441057,2.850772,1.878807,2.543453,-1.074323,5.399666,2.684238,2.371115
2003,AL,Etowah,01055,1983,-7.98,104.23,"Etowah, AL",9864.43,38722.0,1981.0,...,1.696220,-0.126481,4.441057,2.850772,1.878807,2.543453,-1.074323,5.399666,2.684238,2.371115
2004,AL,Jefferson,01073,1983,5.35,155.91,"Jefferson, AL",76684.98,348193.0,1981.0,...,1.696220,-0.126481,4.441057,2.850772,1.878807,2.543453,-1.074323,5.399666,2.684238,2.371115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78702,WY,Sweetwater,56037,2019,1.27,544.73,"Sweetwater, WY",23698.55,27910.0,2017.0,...,-0.144803,1.107718,-2.329085,2.933449,3.581687,0.386010,0.927573,-0.934586,2.954059,2.720550
78703,WY,Teton,56039,2019,9.42,1056.36,"Teton, WY",60437.99,34013.0,2017.0,...,-0.144803,1.107718,-2.329085,2.933449,3.581687,0.386010,0.927573,-0.934586,2.954059,2.720550
78704,WY,Uinta,56041,2019,4.16,248.10,"Uinta, WY",8317.83,11852.0,2017.0,...,-0.144803,1.107718,-2.329085,2.933449,3.581687,0.386010,0.927573,-0.934586,2.954059,2.720550
78705,WY,Washakie,56043,2019,6.05,259.90,"Washakie, WY",3713.75,5087.0,2017.0,...,-0.144803,1.107718,-2.329085,2.933449,3.581687,0.386010,0.927573,-0.934586,2.954059,2.720550


Merge the landfall dataframe with the aggregated economic dataframe

Save as a new csv file titled: 'landfall_bea_hpi_data.csv'

In [21]:
#Merge HPI data with hurricane data
landfall_bea_hpi_df = pd.merge(formatted_landfall_df,aggregated_bea_hpi_df,on=['Year','County','State'],how='inner')

#Save clean df as CSV to directory
landfall_bea_hpi_df.to_csv('Output_CSVs/landfall_bea_hpi_data.csv',index=False)

landfall_bea_hpi_df

,# in Season,Year,Latitude,Longitude,Max Winds (kt),Category at Landfall,Central Prssure (mb),Storm Name,County,State,...,National Jobs AC Mean,National Jobs AC Mean -1,National Jobs AC Mean +1,National Jobs AC Mean +2,National Jobs AC Mean +3,State Jobs AC Mean,State Jobs AC Mean -1,State Jobs AC Mean +1,State Jobs AC Mean +2,State Jobs AC Mean +3
0,3,1983,29.1,-95.1,100,3,962,Alicia,Galveston,TX,...,1.696220,-0.126481,4.441057,2.850772,1.878807,0.073241,2.426554,4.555847,4.271439,-1.943088
1,5,1985,30.4,-89.2,100,3,959,Elena,Harrison,MS,...,2.850772,4.441057,1.878807,3.008315,3.041203,4.169243,5.597101,3.582680,2.653948,2.664073
2,9,1985,40.6,-73.3,75,1,961,Gloria,Suffolk,NY,...,2.850772,4.441057,1.878807,3.008315,3.041203,3.153399,3.953716,2.840822,0.941134,3.198602
3,13,1985,30.0,-85.4,85,2,967,Kate,Bay,FL,...,2.850772,4.441057,1.878807,3.008315,3.041203,6.453085,7.614235,6.030848,2.122551,5.633299
4,2,1986,29.5,-94.2,75,1,990,Bonnie,Jefferson,TX,...,1.878807,2.850772,3.008315,3.041203,2.365933,-1.943088,4.271439,3.263058,1.286409,0.310697
5,13,1987,25.1,-80.5,65,1,993,Floyd,Monroe,FL,...,3.008315,1.878807,3.041203,2.365933,2.212786,2.122551,6.030848,5.633299,3.572276,2.744892
6,4,1989,29.6,-94.4,70,1,986,Chantal,Chambers,TX,...,2.365933,3.041203,2.212786,0.553979,1.43279,0.310697,1.286409,2.268205,1.923199,1.044248
7,11,1989,32.8,-79.8,120,4,934,Hugo,Charleston,SC,...,2.365933,3.041203,2.212786,0.553979,1.43279,2.493086,3.858077,2.943121,-1.357863,0.659108
8,14,1989,29.2,-95.0,75,1,983,Jerry,Galveston,TX,...,2.365933,3.041203,2.212786,0.553979,1.43279,0.310697,1.286409,2.268205,1.923199,1.044248
9,4,1992,25.5,-80.3,145,5,922,Andrew,Miami-Dade,FL,...,1.432790,0.553979,2.646477,3.333367,2.97802,1.513116,0.599037,3.548588,3.685074,3.825204


Upload Hurricane data by costliness to data frame, rename columns, and convert cost column to a number

Save as a new csv file titled: 'formatted_costliness.csv'

In [22]:
#Upload costliness_csv to notebook and rename columns
costliness_df = pd.read_csv("Resources/costliness.csv")
costliness_df = costliness_df.rename(columns={'Name':'Storm Name','Season':'Year','Category':'Max Category','Adjusted Costs':'Adjusted Costs (Billions)'})

#Format Adjusted Costs column so that it can be purely a float typr
costliness_df['Adjusted Costs (Billions)'] = costliness_df['Adjusted Costs (Billions)'].str.replace('$','',regex=False)
costliness_df['Adjusted Costs (Billions)'] = costliness_df['Adjusted Costs (Billions)'].str.replace('B','',regex=False)
costliness_df['Adjusted Costs (Billions)'] = costliness_df['Adjusted Costs (Billions)'].astype(float)

#Resave as a new csv to Resources
costliness_df.to_csv('Resources/formatted_costliness.csv',index=False)

#show preview
costliness_df.head()

,Storm Name,Year,Max Category,Adjusted Costs (Billions)
0,Frederic,1979,3,7.4
1,Alicia,1983,3,9.4
2,Juan,1985,1,4.3
3,Elena,1985,3,3.8
4,Hugo,1989,4,22.7


Upload the new costliness csv as a dataframe and merge it with the 'landfall_bea_hpi_df' dataframe

Save as a new csv file titled: 'costliness_bea_hpi_data.csv'

In [23]:
#Re-Upload formated landfall and costliness CSVs to notebook
formatted_costliness_df = pd.read_csv("Resources/formatted_costliness.csv")

#Merge the two data frames into one
costliness_bea_hpi_df = pd.merge(landfall_bea_hpi_df,formatted_costliness_df,on = ['Year','Storm Name'],how='inner')

#Drop rows with empty columns
costliness_bea_hpi_df = costliness_bea_hpi_df.dropna()

#Save new csv to avoid long run time of above code 
costliness_bea_hpi_df.to_csv('Output_CSVs/costliness_bea_hpi_data.csv',index=False)

#Show dataframe preview
costliness_bea_hpi_df

,# in Season,Year,Latitude,Longitude,Max Winds (kt),Category at Landfall,Central Prssure (mb),Storm Name,County,State,...,National Jobs AC Mean +1,National Jobs AC Mean +2,National Jobs AC Mean +3,State Jobs AC Mean,State Jobs AC Mean -1,State Jobs AC Mean +1,State Jobs AC Mean +2,State Jobs AC Mean +3,Max Category,Adjusted Costs (Billions)
0,3,1983,29.1,-95.1,100,3,962,Alicia,Galveston,TX,...,4.441057,2.850772,1.878807,0.073241,2.426554,4.555847,4.271439,-1.943088,3,9.4
1,5,1985,30.4,-89.2,100,3,959,Elena,Harrison,MS,...,1.878807,3.008315,3.041203,4.169243,5.597101,3.582680,2.653948,2.664073,3,3.8
2,11,1989,32.8,-79.8,120,4,934,Hugo,Charleston,SC,...,2.212786,0.553979,1.43279,2.493086,3.858077,2.943121,-1.357863,0.659108,4,22.7
3,4,1992,25.5,-80.3,145,5,922,Andrew,Miami-Dade,FL,...,2.646477,3.333367,2.97802,1.513116,0.599037,3.548588,3.685074,3.825204,5,60.2
4,17,1995,30.3,-87.1,100,3,942,Opal,Escambia,FL,...,1.979274,2.310141,2.04344,3.825204,3.685074,3.714625,3.793512,3.239556,3,9.6
5,6,1996,33.9,-78.0,100,3,954,Fran,Brunswick,NC,...,2.310141,2.04344,1.571721,1.726423,3.643462,3.002345,1.779732,1.750796,3,10.0
6,7,1998,24.5,-81.8,90,2,981,Georges,Monroe,FL,...,1.571721,1.768222,-0.798434,3.239556,3.793512,2.003655,2.899257,1.599120,2,11.5
7,7,1998,30.4,-88.9,90,2,964,Georges,Harrison,MS,...,1.571721,1.768222,-0.798434,1.818317,1.858989,1.709210,0.523601,-1.789630,2,11.5
8,8,1999,33.8,-78.0,90,2,956,Floyd,Brunswick,NC,...,1.768222,-0.798434,-0.163517,1.750796,1.779732,1.128351,-2.189951,-0.275266,2,12.1
9,13,2003,34.9,-76.2,90,2,957,Isabel,Carteret,NC,...,1.485947,1.816113,1.578675,0.057359,-0.275266,2.107865,1.922273,2.144557,2,9.3


clean_new_hpi_income_df = Total HPI data with new columns for every county

clean_hurricane_hpi_income_df = Comined previous two data frames. All hurricane strikes with HPI data.